In [ ]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa
import dsds.fs as fs
import dsds.transform as t

In [11]:
df = pl.read_csv("../data/train.csv")
print(df.shape)
df.head(2)

(120000, 3)


Class Index,Title,Description
i64,str,str
3,"""Wall St. Bears…","""Reuters - Shor…"
3,"""Carlyle Looks …","""Reuters - Priv…"


In [12]:
import dsds.text as tt
tt.count_vectorizer(df, "Description", min_dfreq=0.02).head(2)

Class Index,Title,Description::cnt_2004,Description::cnt_accord,Description::cnt_afp,Description::cnt_american,Description::cnt_announc,Description::cnt_attack,Description::cnt_back,Description::cnt_base,Description::cnt_billion,Description::cnt_busi,Description::cnt_call,Description::cnt_citi,Description::cnt_com,Description::cnt_compani,Description::cnt_comput,Description::cnt_corp,Description::cnt_could,Description::cnt_countri,Description::cnt_day,Description::cnt_deal,Description::cnt_elect,Description::cnt_end,Description::cnt_european,Description::cnt_expect,Description::cnt_feder,Description::cnt_final,Description::cnt_first,Description::cnt_five,Description::cnt_forc,Description::cnt_former,Description::cnt_four,Description::cnt_friday,Description::cnt_game,Description::cnt_get,Description::cnt_giant,…,Description::cnt_second,Description::cnt_secur,Description::cnt_servic,Description::cnt_set,Description::cnt_share,Description::cnt_show,Description::cnt_softwar,Description::cnt_start,Description::cnt_state,Description::cnt_stock,Description::cnt_sunday,Description::cnt_system,Description::cnt_take,Description::cnt_target,Description::cnt_team,Description::cnt_technolog,Description::cnt_third,Description::cnt_three,Description::cnt_thursday,Description::cnt_time,Description::cnt_today,Description::cnt_top,Description::cnt_trade,Description::cnt_tuesday,Description::cnt_two,Description::cnt_unit,Description::cnt_use,Description::cnt_victori,Description::cnt_washington,Description::cnt_wednesday,Description::cnt_week,Description::cnt_win,Description::cnt_world,Description::cnt_would,Description::cnt_year,Description::cnt_yesterday,Description::cnt_york
i64,str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
3,"""Wall St. Bears…",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,"""Carlyle Looks …",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
tt.get_word_cnt_table(df, "Description", min_dfreq=0.02)

stemmed,Description,doc_freq
str,list[str],f32
"""2004""","[""2004""]",0.02595
"""accord""","[""accordingly"", ""according"", … ""accords""]",0.021425
"""afp""","[""afp""]",0.021158
"""american""","[""americans"", ""americanized"", … ""american""]",0.026458
"""announc""","[""announcer"", ""announcing"", … ""announc""]",0.036667
"""attack""","[""attacking"", ""attacker"", … ""attacked""]",0.022292
"""back""","[""backe"", ""back"", … ""backing""]",0.031142
"""base""","[""basing"", ""based"", … ""base""]",0.020458
"""billion""","[""billion"", ""billions""]",0.024275


In [ ]:
from dsds._rust import rs_cnt_vectorizer, rs_get_word_cnt_table, rs_snowball_stem, rs_levenshtein_dist
import dsds.text as tt
c = "Ad Topic Line"

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = df["Description"].to_list()

In [15]:
vectorizer = CountVectorizer(max_df=0.95, min_df=0.02)
X = np.array(vectorizer.fit_transform(corpus).todense())
cols = vectorizer.get_feature_names_out().tolist()
df_words = pl.from_numpy(X, schema=cols)
df_combined = pl.concat([df, df_words], how="horizontal")
df_combined

Class Index,Title,Description,10,2004,39,about,according,afp,after,against,all,american,an,and,announced,ap,are,as,at,back,be,been,before,billion,business,but,by,can,city,co,com,company,computer,corp,could,country,…,than,that,the,their,they,third,this,three,thursday,time,to,today,top,tuesday,two,united,up,us,victory,was,washington,wednesday,week,were,what,when,which,who,will,win,with,world,would,year,years,yesterday,york
i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
3,"""Wall St. Bears…","""Reuters - Shor…",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"""Carlyle Looks …","""Reuters - Priv…",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"""Oil and Econom…","""Reuters - Soar…",0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"""Iraq Halts Oil…","""Reuters - Auth…",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,…,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"""Oil prices soa…","""AFP - Tearaway…",0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,"""Stocks End Up,…","""Reuters - Stoc…",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,…,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,"""Money Funds Fe…","""AP - Assets of…",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,…,0,0,3,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"""Fed minutes sh…","""USATODAY.com -…",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,…,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"""Safety Net (Fo…","""Forbes.com - A…",0,0,0,1,0,0,2,0,0,0,1,0,0,0,0,1,3,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,…,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
cols

In [ ]:
df_combined["system"].sum()

In [ ]:
df2["Ad Topic Line::cnt_system"].sum()

In [ ]:
df2["Ad Topic Line::cnt_analyz"].sum()

In [ ]:
df_words = pl.from_numpy(X, schema=cols)
df_words

In [ ]:
df = pl.read_csv("../data/advertising.csv")
df.head()

In [ ]:
import dsds.metrics as me
from sklearn.metrics import log_loss

def pure_numpy_logloss(y_actual:np.ndarray, y_predict:np.ndarray):
    return -np.mean(y_actual * np.log(y_predict) + (1 - y_actual) * np.log(1 - y_predict))

pred = np.random.random(size=500_000) # Some random fake predictions
actual = np.round(np.random.random(size=500_000)).astype(np.int8) # Some random fake actual labels
# Yielding the same result up to 12 digits
print(round(me.logloss(actual, pred), 12) == round(log_loss(actual, pred), 12))
print(round(me.logloss(actual, pred), 12) == round(pure_numpy_logloss(actual, pred), 12))

In [ ]:
%%timeit
me.logloss(actual, pred) # dsds

In [ ]:
%%timeit
log_loss(actual, pred) # sklearn

In [ ]:
%%timeit
pure_numpy_logloss(actual, pred) # pure numpy

In [ ]:
# from sklearn.datasets import make_classification
# orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# # This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# # You must turn other dataframe formats into Polars for dsds to work.
# df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# # Turn it into Pandas.
# df_pd = df.to_pandas()
# target = "target"
# features = df.columns
# features.remove(target)

In [ ]:
df = pl.DataFrame({
    "a": [1, np.nan, None],
    "b": [1,2,3]
})
df

In [ ]:
ps.invalid_inferral(df, threshold=0.5, include_null=True)

In [ ]:
ps.discrete_inferral(df)

In [ ]:
temp = ps.get_unique_count(df.with_row_count(offset=1).set_sorted("row_nr"), include_null_count=True)
len_df = temp.filter(pl.col("column") == "row_nr").item(0,1)
print(len_df)
temp

In [ ]:
from sklearn.datasets import make_classification
orig_x, orig_y = make_classification(n_samples = 50_000, n_features = 50, n_informative = 20, n_redundant = 30)
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
features = df_pl.columns
features.remove("target")

In [ ]:
for train, test in sa.time_series_split(df_pl, n_splits=5, offset = 1000):
    print(train.shape)
    print(test.shape)

In [ ]:
%%timeit
fs.discrete_ig(df_pl, target="target", cols=features)

In [ ]:
%%timeit
fs.discrete_ig2(df_pl, target="target", cols=features)

In [ ]:

me.logloss(actual, pred, check_binary=False)

In [ ]:
import nltk
from nltk.corpus import stopwords